In [397]:
# url to first pages
# enter cian.ru and avito.ru urls of desired city and a name of the city
cian_url = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"
# avito_url = "https://www.avito.ru/sankt-peterburg/kvartiry?s_trg=1"
avito_url = 'https://www.avito.ru/sankt-peterburg/kvartiry/prodam?f=549_5695-5696-5697-5698-5699-5700-5701-11018-11019-11020-11021&s_trg=4'
city = "Санкт-Петербург"

#example of samara urls
# avito_url = 'https://www.avito.ru/samara/kvartiry?s_trg=3'
# cian_url = 'https://samara.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=4966&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1'
# city = 'Самара'

api_key = '800e1c1a-3d68-496e-88a9-009b98d7eb75'
api_url = 'https://geocode-maps.yandex.ru/1.x/?apikey=' + api_key + '&format=json&geocode=' + city + ',+'

In [ ]:
# необходимо сначала исполнить следующую ячейку, чтобы загрузить все функции
# работает долго

# avito to csv
print('start avito parsing, url = ', avito_url)
avito_flats = parse_avito(avito_url)
write_to_csv(avito_flats, 'spb_test_avito.csv')
print('finished avito parsing')

# cian to csv
print('start cian parsing, url = ', cian_url)
cian_flats = parse_cian(cian_url)
write_to_csv(cian_flats, 'spb_test_cian.csv')
print('finished cian parsing')

# merge
print('merging cian and avito data')
merge('spb_test_avito.csv', 'spb_test_cian.csv', 'spb_test_merged.csv')

# geotag
print('geotag all data')
geotag('spb_test_merged.csv', 'spb_test_geotagged.csv')

In [433]:
# execute this cell to load all neccessary functions

from bs4 import BeautifulSoup
import urllib.request
import requests
import re

#write collected data to .csv file as a pandas dataframe
import numpy
import pandas as pd 

#decode geotagging answer
import json
import csv

#avito
#extract price
def price_from_json(flat):
    price = flat.find("div", {"class": "popup-prices"})
    if price != None:
        price = price['data-prices']
    else:
        return -1
    price_ind = price.find('RUB":')
    price_cut = price[price_ind + 5:]
    price_in_rub = price_cut.split(",")[0]
    return int(price_in_rub)

#extract all relevant data from single page
def parse_page_avito(flats, data_flats):
    rent_count = 0
    print("flats on page: ", len(flats))
    for (index, flat) in enumerate(flats):

        print("Flat number", index)

        #price
        print("price:")
        price = price_from_json(flat)
        if(price < 300_000):
            rent_count += 1
            print("Rent found, skip. price = ", price)
            continue
        print(price)
        print()


        #rooms, area, floor level
        print("rooms:")
        description = flat.find("a", {"class": "item-description-title-link"})['title']
        try_rooms = description.split(",")[0][0]
        if try_rooms.isnumeric():
            rooms = int(try_rooms)
        else:
            rooms = 1
        print(rooms)
        print()

        print("area:")
        area = float(description.split(",")[1].split(" ")[1])
        print(area, "m^2")
        print()

        print(description)

        print("floor:")
        floor = description.split(",")[2].split(" ")[1].split("/")[0]
        print('floors total = ', description)
        floors_total = description.split(",")[2].split(" ")[1].split("/")[1]
        print("" + floor + ", total floors: " + floors_total)
        print()

        #subway, distance to subway, adress
        full_adress = flat.find("p", {"class": "address"}).getText()

        adress = ','.join(full_adress.split(",")[1:])
        print("adress:")
        print(adress)
        print()

        subway = full_adress.split(",")[0]
        for (ind,el) in enumerate(subway.split()):
            try:
                float(el)
                subway_station = ' '.join(subway.split()[:ind])
                distance = float(subway.split()[ind])
            except ValueError:
                pass

        print("subway:")
        print(subway_station)
        print()

        if distance < 99:
            distance *= 1000
        print("distance to subway:")
        print(distance, "meters")
        print()

        print("go to next page")
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway_station, distance))
    
    print(rent_count)
    return data_flats


def parse_avito(avito_url):
    avito_real_estate = urllib.request.urlopen(avito_url).read()
    #parse avito with beautiful soup
    avito_soup = BeautifulSoup(avito_real_estate)
    # url_base = "https://www.avito.ru"
    flats = avito_soup.findAll("div", {"class":"item_table-description"})
    page_count = 0

    data_flats = parse_page_avito(flats, [])

    while True:
        try:
            page_count += 1
            print("Page: ", page_count)
            next_page = avito_soup.find("a", {"class":"pagination-page"}).get('href')
            flats = avito_soup.findAll("div", {"class":"item_table-description"})
            data_flats = parse_page_avito(flats, data_flats)

            if page_count > 100:
                print('page count > 100, break')
                break
        except Error as error:
            print(error, ' error, break')
            break

    print("finish, page count = ", page_count)
    print("data len: ", len(data_flats))
    new_data_flats = []
    for flat in data_flats:
        new_flat = (flat[0],flat[1],flat[2],flat[3],flat[4],flat[5].replace('\xa0', '', 2),flat[6],flat[7])
        new_data_flats.append(new_flat)
    
    return new_data_flats

def write_to_csv(new_data_flats, file_name):
    
    a = numpy.asarray(new_data_flats)
    pd.DataFrame(a).to_csv(file_name, header=['price', 'rooms', 'area', 'floor', 'floors_total', 'adress',
                                           'subway_station', 'distance to subway'])

#cian
    
def next_page_cian(first_page, soup):
    next_p = soup.find("li", {"class": re.compile('^.*list-item--active.*')})
    if next_p and next_p.nextSibling:
        return next_p.nextSibling.a['href']
    else:
        return -1

def parse_cian_page(cian_soup, data_flats):
    counter = 0
    flats = cian_soup.findAll("div", {"class": "c6e8ba5398--info-container--YhZ2y"})
    print('flats = ',len(flats))
    for flat in flats:
        
        counter += 1
        # rooms, area, floor
        addr = flat.find("div", {"class": re.compile('^.*title.*')})
        if addr:
            try:
                rooms = int(addr.text[0])
                print('rooms = ', rooms)
            except Exception:
                rooms = 1
            try:
                area = float(addr.text.split(',')[1].split()[0])
            except Exception as error:
                print(error,', area error, area = ', addr.text.split(',')[1].split()[0])
                continue
            print('full addr = ', addr.text)
            print('area = ', area)
            floor = int(addr.text.split(',')[2].split(' ')[1].split('/')[0])
            print('floors = ', floor)
            try:
                floors_total = int(addr.text.split(',')[2].split(' ')[1].split('/')[1])
            except Exception as error:
                print('floors total error = ', error)
            print('floors total = ', floors_total)
            
            print(addr.text)

        subway = flat.find("div", {"class": re.compile('^.*underground-name.*')})
        if subway:
            subway = subway.text
            print(subway)
        else:
            subway = None

        remoteness = flat.find("div", {"class": re.compile('^.*remoteness.*')})
        if remoteness:
            print(remoteness.text)
            if remoteness.text.split('\xa0')[2] == 'пешком':
                remoteness = int(remoteness.text.split('\xa0')[0]) * 100
            else:
                remoteness = int(remoteness.text.split('\xa0')[0]) * 1000
            print('distance to subway = ',remoteness)
        else:
            remoteness = None
            
        adress = flat.findAll("a", {"class": re.compile('^.*address.*')})
        if adress:
            if(len(adress) > 5):
                adress = adress[4].text+ ', ' + adress[5].text
                print(adress)
            else:
                continue
                
        price = flat.find("div", {"class": re.compile('^.*price.*')}).find("div",{'class': re.compile('.*header.*')})
        if price:
            print(price.text)
            price_array = price.text.split(' ')[:len(price.text.split(' ')) - 1]
            price = 0
            for el in price_array:
                price *= 1000
                price += int(el)
            print('price = ', price)
            
        data_flats.append((price, rooms, area, floor, floors_total, adress, subway, remoteness))
        print()
    return data_flats

def parse_cian(cian_url):
    base_url = cian_url.split('cian.ru')[0] + 'cian.ru' 
    cian_real_estate = urllib.request.urlopen(cian_url).read()
    cian_soup = BeautifulSoup(cian_real_estate)
    counter = 0
    flats = []
    pages = 58
    while counter < pages:
        counter += 1
        flats = parse_cian_page(cian_soup, flats)
        cian_url = next_page_cian(cian_url, cian_soup)
        if cian_url == -1:
            break
        if base_url not in cian_url:
            cian_url = base_url + cian_url
        print("url = ", cian_url)
        cian_html = urllib.request.urlopen(cian_url).read()
        cian_soup = BeautifulSoup(cian_html)
        print("counter = ", counter)
    return flats
        
        
#add geotagging to existing csv file
def geotag(file_in, file_out):
    with open(file_in) as csv_file, open(file_out, 'w') as out:

        writer = csv.writer(out, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        csv_reader = csv.reader(csv_file, delimiter=',')
        for ind, row in enumerate(csv_reader):
            print('row ', ind)
            if ind == 0:
                writer.writerow((*row, 'longitude', 'latitude'))

            else:
                #TODO replace spaces with '+'
                print('requested address = ', row[6].replace(' ','+'))
                print('full url = ',api_url + row[6].replace(' ','+') )
                r = requests.get(api_url + row[6].replace(' ','+'))
                d = json.loads(r.text)
                if d['response']['GeoObjectCollection']['metaDataProperty']['GeocoderResponseMetaData']['found'] != 0:
                    longitude, latitude = d['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
                    print('long = ', longitude, ', latitude = ', latitude)
                    writer.writerow((*row, longitude, latitude))
                    out.flush()
                else:
                    print("geoposition not found, row skipped")
                
#merge two data files
def merge(first, second, out):
    
    with open(first) as cian, open(second) as avito, open(out, 'w') as out:
        writer = csv.writer(out, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        cian_reader = csv.reader(cian, delimiter=',')
        avito_reader = csv.reader(avito, delimiter =',')

        for row in avito_reader:
            writer.writerow(row)

        for row in cian_reader:
            writer.writerow(row)

In [202]:
#flat data class combining all relevant data
class Flat:
    
    def __init__(self, flat_price, flat_rooms, flat_area, flat_floor, 
                 flat_floors_total, flat_address, flat_subway_st, flat_distance_to_subway):
        self.flat_price = flat_price
        self.flat_rooms = flat_rooms 
        self.flat_area = flat_area
        self.flat_floor = flat_floor
        self.flat_floors_total = flat_floors_total
        self.flat_address = flat_address
        self.flat_subway_st = flat_subway_st
        self.flat_distance_to_subway = flat_distance_to_subway
        
    def __str__(self):
        return ''.join("adress = "+ flat_address + 
                       ", price = "+ str(flat_price)+ 
                       ", rooms = "+ str(flat_rooms)+
                        ", floor = "+ str(flat_floor)+ 
                       ", area = " + str(flat_area) +
                      ", total_floors = "+ str(flat_floors_total)+ 
                       ", subway = "+ flat_subway_st + 
                       ", distance to subway = " + flat_distance_to_subway)

In [220]:
#
subways = set()
for flat in data_flats:
    subways.add(flat[6])
print(subways)

{'Чкаловская', 'Проспект Просвещения', 'Пионерская', 'Пролетарская', 'Приморская', 'Купчино', 'Проспект Большевиков', 'Елизаровская', 'Ломоносовская', 'Комендантский проспект', 'Василеостровская', 'Ладожская', 'Девяткино', 'Спортивная', 'Московская', 'Чернышевская', 'Автово', 'Звездная', 'Обухово', 'Парнас', 'Достоевская', 'Проспект Ветеранов', 'Новочеркасская', 'Гражданский проспект'}


In [399]:
cian_url

'https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'

 Парс сайта cian.ru

In [154]:
#read page, load to soup object
cian_real_estate = urllib.request.urlopen("https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1").read()
cian_soup = BeautifulSoup(cian_real_estate)

In [264]:
# cian_url = "https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"

base_url = samara_cian_url.split('cian.ru')[0] + 'cian.ru' 
cian_real_estate = urllib.request.urlopen(samara_cian_url).read()
cian_soup = BeautifulSoup(cian_real_estate)
cian_data = cian_flats(cian_soup, [])

28
rooms =  1
area =  37.0
floors =  11
floors total =  16
1-комн. кв., 37 м², 11/16 этаж
Старая Деревня
20 минут пешком
distance to subway =  2000
Стародеревенская улица, 18
5 500 000 ₽
price =  5500000

rooms =  1
area =  55.0
floors =  4
floors total =  14
1-комн. кв., 55 м², 4/14 этаж
Площадь Ленина
10 минут на транспорте
distance to subway =  10000
улица Жукова, 1
7 547 000 ₽
price =  7547000

rooms =  2
area =  58.0
floors =  18
floors total =  25
2-комн. кв., 58 м², 18/25 этаж
Комендантский проспект
30 минут пешком
distance to subway =  3000
rooms =  3
area =  181.0
floors =  10
floors total =  10
3-комн. кв., 181 м², 10/10 этаж
Озерки
5 минут на транспорте
distance to subway =  5000
Афанасьевская улица, 1
18 960 000 ₽
price =  18960000

rooms =  3
area =  86.0
floors =  10
floors total =  11
3-комн. кв., 86 м², 10/11 этаж
Площадь Восстания
10 минут пешком
distance to subway =  1000
Кременчугская улица, 17к3
10 495 000 ₽
price =  10495000

rooms =  1
area =  37.0
floors =  14
fl

In [344]:

    
    
    print('finished parsing')
a = numpy.asarray(flats)
pd.DataFrame(a).to_csv("samara_cian_data.csv", header=['price', 'rooms', 'area', 'floor', 'floors_total', 'adress',
                                           'subway_station', 'distance to subway'])

flats =  28
area =  52.0
floors =  17
floors total =  18
Евродвушка в ЖК Парк Победы1-комн. кв., 52 м², 17/18 этаж
Спортивная
13 минут пешком
distance to subway =  1300
Аэродромная улица, 98А
4 700 000 ₽
price =  4700000

rooms =  2
area =  45.0
floors =  5
floors total =  5
2-комн. кв., 45 м², 5/5 этаж
Кировская
24 минуты пешком
distance to subway =  2400
м. Кировская, улица Свободы
3 200 000 ₽
price =  3200000

rooms =  4
area =  209.0
floors =  16
floors total =  25
4-комн. кв., 209 м², 16/25 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors 

counter =  3
flats =  28
area =  52.0
floors =  17
floors total =  18
Евродвушка в ЖК Парк Победы1-комн. кв., 52 м², 17/18 этаж
Спортивная
13 минут пешком
distance to subway =  1300
Аэродромная улица, 98А
4 700 000 ₽
price =  4700000

rooms =  2
area =  49.0
floors =  3
floors total =  24
2-комн. кв., 49 м², 3/24 этаж
rooms =  2
area =  78.0
floors =  7
floors total =  17
2-комн. кв., 78 м², 7/17 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subway

counter =  6
flats =  28
rooms =  2
area =  92.0
floors =  3
floors total =  16
2-комн. кв., 92 м², 3/16 этаж
rooms =  2
area =  80.0
floors =  21
floors total =  24
2-комн. кв., 80 м², 21/24 этаж
rooms =  2
area =  68.0
floors =  9
floors total =  15
2-комн. кв., 68 м², 9/15 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subway =  1600
улица Тухачевского, 80
3 500 000 ₽
price =  3500000

rooms =  2
area =  94.0
floors =  8
floors total =  25
2-комн

counter =  9
flats =  28
rooms =  1
area =  44.0
floors =  3
floors total =  24
1-комн. кв., 44 м², 3/24 этаж
Безымянка
25 минут на транспорте
distance to subway =  25000
м. Безымянка, Московское шоссе
2 220 000 ₽
price =  2220000

rooms =  1
area =  56.0
floors =  19
floors total =  25
1-комн. кв., 56 м², 19/25 этаж
rooms =  2
area =  75.0
floors =  16
floors total =  18
2-комн. кв., 75 м², 16/18 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subwa

counter =  12
flats =  28
rooms =  1
area =  55.0
floors =  8
floors total =  23
1-комн. кв., 55 м², 8/23 этаж
rooms =  2
area =  45.0
floors =  5
floors total =  5
2-комн. кв., 45 м², 5/5 этаж
Кировская
24 минуты пешком
distance to subway =  2400
м. Кировская, улица Свободы
3 200 000 ₽
price =  3200000

rooms =  2
area =  80.0
floors =  21
floors total =  24
2-комн. кв., 80 м², 21/24 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subway =  1600
ули

...

counter =  45
flats =  28
rooms =  2
area =  78.0
floors =  7
floors total =  17
2-комн. кв., 78 м², 7/17 этаж
rooms =  3
area =  131.0
floors =  2
floors total =  10
3-комн. кв., 131 м², 2/10 этаж
area =  35.0
floors =  2
floors total =  24
Студия, 35 м², 2/24 этаж
Безымянка
25 минут на транспорте
distance to subway =  25000
м. Безымянка, Московское шоссе
1 960 000 ₽
price =  1960000

rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subway =  1600
улица Т

counter =  48
flats =  28
rooms =  2
area =  49.0
floors =  3
floors total =  24
2-комн. кв., 49 м², 3/24 этаж
rooms =  2
area =  45.0
floors =  5
floors total =  5
2-комн. кв., 45 м², 5/5 этаж
Кировская
24 минуты пешком
distance to subway =  2400
м. Кировская, улица Свободы
3 200 000 ₽
price =  3200000

rooms =  2
area =  66.0
floors =  2
floors total =  4
2-комн. кв., 66 м², 2/4 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subway =  1600
улица Т

counter =  51
flats =  28
area =  86.0
floors =  19
floors total =  25
Для счастливой семьи3-комн. кв., 86 м², 19/25 этаж
rooms =  2
area =  70.0
floors =  19
floors total =  26
2-комн. кв., 70 м², 19/26 этаж
rooms =  2
area =  68.0
floors =  9
floors total =  15
2-комн. кв., 68 м², 9/15 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subway =  1600
улица Тухачевского, 80
3 500 000 ₽
price =  3500000

rooms =  2
area =  94.0
floors =  8
floors total 

counter =  54
flats =  28
rooms =  2
area =  63.0
floors =  18
floors total =  22
2-комн. кв., 63 м², 18/22 этаж
rooms =  3
area =  89.0
floors =  9
floors total =  25
3-комн. кв., 89 м², 9/25 этаж
rooms =  1
area =  46.0
floors =  8
floors total =  9
1-комн. кв., 46 м², 8/9 этаж
rooms =  2
area =  42.0
floors =  1
floors total =  3
2-комн. кв., 42 м², 1/3 этаж
Центральная улица, 2
1 550 000 ₽
price =  1550000

rooms =  1
area =  44.0
floors =  1
floors total =  17
1-комн. кв., 44 м², 1/17 этаж
Российская
7 минут на транспорте
distance to subway =  7000
rooms =  2
area =  44.0
floors =  5
floors total =  5
2-комн. кв., 44 м², 5/5 этаж
Советская
5 минут пешком
distance to subway =  500
улица Гагарина, 135
2 600 000 ₽
price =  2600000

rooms =  1
area =  50.0
floors =  14
floors total =  16
1-комн. кв., 50 м², 14/16 этаж
Московская
16 минут пешком
distance to subway =  1600
улица Тухачевского, 80
3 500 000 ₽
price =  3500000

rooms =  2
area =  94.0
floors =  8
floors total =  25
2-комн.

counter =  57
flats =  25
rooms =  1
area =  38.0
floors =  11
floors total =  22
1-комн. кв., 38 м², 11/22 этаж
rooms =  1
area =  31.0
floors =  5
floors total =  5
1-комн. кв., 31 м², 5/5 этаж
rooms =  1
area =  31.0
floors =  4
floors total =  4
1-комн. апарт., 31 м², 4/4 этаж
rooms =  2
area =  56.0
floors =  2
floors total =  4
2-комн. кв., 56 м², 2/4 этаж
проспект Кирова, 61
1 880 000 ₽
price =  1880000

rooms =  1
area =  34.0
floors =  14
floors total =  15
1-комн. кв., 34 м², 14/15 этаж
rooms =  1
area =  32.0
floors =  3
floors total =  5
1-комн. кв., 32 м², 3/5 этаж
улица Победы, 129
1 850 000 ₽
price =  1850000

rooms =  1
area =  35.0
floors =  24
floors total =  25
1-комн. кв., 35 м², 24/25 этаж
Московская
rooms =  1
area =  35.0
floors =  3
floors total =  19
1-комн. кв., 35 м², 3/19 этаж
rooms =  3
area =  62.0
floors =  5
floors total =  5
3-комн. кв., 62 м², 5/5 этаж
улица Фестивальная, 3
1 850 000 ₽
price =  1850000

rooms =  2
area =  46.0
floors =  1
floors total 

In [297]:
r = requests.get('https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Санкт-Петербург,+улица+Жукова,+1')
d = json.loads(r.text)
longitude, latitude = d['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()

('30.405062', '59.962435')

In [371]:


#geotag()

In [372]:

# merge('samara_avito.csv', 'samara_cian_data.csv', 'samara_merged.csv')
geotag('samara_merged.csv', 'samara_geotagged.csv')

row  0
row  1
row  2
row  3
row  4
row  5
row  6
row  7
row  8
row  9
row  10
row  11
row  12
row  13
row  14
row  15
row  16
row  17
row  18
row  19
row  20
row  21
row  22
row  23
row  24
row  25
row  26
row  27
row  28
row  29
row  30
row  31
row  32
row  33
row  34
row  35
row  36
row  37
row  38
row  39
row  40
row  41
row  42
row  43
row  44
row  45
row  46
row  47
row  48
row  49
row  50
row  51
row  52
row  53
row  54
row  55
row  56
row  57
row  58
row  59
row  60
row  61
row  62
row  63
row  64
row  65
row  66
row  67
row  68
row  69
row  70
row  71
row  72
row  73
row  74
row  75
row  76
row  77
row  78
row  79
row  80
row  81
row  82
row  83
row  84
row  85
row  86
row  87
row  88
row  89
row  90
row  91
row  92
row  93
row  94
row  95
row  96
row  97
row  98
row  99
row  100
row  101
row  102
row  103
row  104
row  105
row  106
row  107
row  108
row  109
row  110
row  111
row  112
row  113
row  114
row  115
row  116
row  117
row  118
row  119
row  120
row  121
row  122
row

long =  50.284276 , latitude =  53.207329
row  684
requested address =  +Россия,+Самарская+область,+Самара,+Аэродромная+ул,+98а+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++Россия,+Самарская+область,+Самара,+Аэродромная+ул,+98а+
long =  50.206275 , latitude =  53.194438
row  685
requested address =  +Самара,+улица+Георгия+Димитрова,+108+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++Самара,+улица+Георгия+Димитрова,+108+
long =  50.223757 , latitude =  53.260328
row  686
requested address =  +улСвободы,+145+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++улСвободы,+145+
long =  50.267495 , latitude =  53.225463
row  687
requested address =  +Московское+ш.,+стр.1+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json

long =  50.284276 , latitude =  53.207329
row  714
requested address =  +Россия,+Самарская+область,+Самара,+Аэродромная+ул,+98а+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++Россия,+Самарская+область,+Самара,+Аэродромная+ул,+98а+
long =  50.206275 , latitude =  53.194438
row  715
requested address =  +Самара,+улица+Георгия+Димитрова,+108+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++Самара,+улица+Георгия+Димитрова,+108+
long =  50.223757 , latitude =  53.260328
row  716
requested address =  +улСвободы,+145+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++улСвободы,+145+
long =  50.267495 , latitude =  53.225463
row  717
requested address =  +Московское+ш.,+стр.1+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json

long =  50.284276 , latitude =  53.207329
row  744
requested address =  +Россия,+Самарская+область,+Самара,+Аэродромная+ул,+98а+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++Россия,+Самарская+область,+Самара,+Аэродромная+ул,+98а+
long =  50.206275 , latitude =  53.194438
row  745
requested address =  +Самара,+улица+Георгия+Димитрова,+108+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++Самара,+улица+Георгия+Димитрова,+108+
long =  50.223757 , latitude =  53.260328
row  746
requested address =  +улСвободы,+145+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json&geocode=Самара,++улСвободы,+145+
long =  50.267495 , latitude =  53.225463
row  747
requested address =  +Московское+ш.,+стр.1+
full url =  https://geocode-maps.yandex.ru/1.x/?apikey=800e1c1a-3d68-496e-88a9-009b98d7eb75&format=json

KeyboardInterrupt: 

In [365]:
empty_json = '{"response":{"GeoObjectCollection":{"metaDataProperty":{"GeocoderResponseMetaData":{"request":"Самара, пос. Соцгород, переулок Ново-Молодежный","found":"0","results":"10"}},"featureMember":[]}}}'
d = json.loads(empty_json)
d['response']['GeoObjectCollection']['metaDataProperty']['GeocoderResponseMetaData']['found'] 

'0'